# N皇后问题

规则很简单：将 n 个皇后放置在 n×n 的棋盘上，并且使皇后彼此之间不能相互攻击。（即：任何两个皇后都不能处于同一条横行、纵行或斜线上。）

![国际象棋棋盘](https://assets.leetcode-cn.com/aliyun-lc-upload/uploads/2018/10/12/8-queens.png)

要求：算法给出所有 N 皇后的可行解决方案。

# 解决思路

## 提取已知信息

1. 确定了行和列即可确定棋子的位置。
2. 棋盘每一行只能放置一个皇后。
3. 棋盘每一列只能放置一个皇后。

## 确定数据结构

首先很自然的想到**使用二维数组来描述棋盘**，数组内使用 0/1 来表示有无棋子。
这种数据结构可以表示的棋子组合数为：$C_{N^2}^N$，是非常多的。

由于**已知每行放置一个皇后**，所以可以对棋盘进行降维处理，**使用一维数组 $row[]$ 表示棋盘的每一行**，数组内使用 $[0, N)$ 的整数来表示皇后放置在那一列。
这样即保证了每行只会出现一个皇后，又降低了组合数：$A_N^N$

由于任意皇后不能出现在同一列，我们需要在处理的过程中**使用一维数组 $col[]$ 记录每一列中是否有棋子**。

## 一些猜想

在 N 不是很大的情况下，可以枚举 N! 种组合，对每种解的合法性进行检测，并输出合法解。

那么如何检测解的合法性呢？其实只需要检测新增加的棋子是否与已放置的棋子在一条斜线上即可：棋子1坐标（$x_1$, $y_1$），棋子2坐标（$x_2$, $y_2$），即检测等式 $(x_2 - x_1) = \pm (y_2 - y_1)$ 是否成立。

## 算法实现

穷举并检测的方案无非两种实现方法，即循环和递归。
从前面的推理可以得知，**所以可能组合的情况是阶乘级别的**，这种结构需要 N 层循环才能描述清楚，所以只能使用递归的方案。

+ 这里简单介绍一下递归算法的实现注意点⚠️：
    + 递归函数会进行自我调用；
    + 在调用点之前所修改的部分数据会作为形参传递给下一层调用。
        + 如果是传值则不需要考虑恢复上下文，但可能会影响效率。
        + 如果是传引用则需要考虑调用点后文数据状态的恢复。
    + 递归需要有一个终止条件以避免死循环导致堆栈溢出。
    + 通常需要一个外部函数的调用来触发递归。

### 时间复杂度

在放置棋子时考虑行列错开，那么共有 $N!$ 种组合，如果在得到一个完整的放置方案时才检查棋子是否在斜线上，那个每种组合需要 $O(N^2)$ 时间复杂度的验证时间。
但如果在每行放置棋子时就立刻检测是否和其他棋子在一条斜线上，那么验证一个部分放置方案的时间复杂度为 $O(N)$，那么算法的整体复杂度可降低为 $O(N^3)$。

### 空间复杂度

+ 如果不需要保存解，那么只要 $O(N)$ 的空间来保存中间数组。
+ 如果需要保存所有的解，那么需要 $O(N^3)$ 的空间来保存所有可行方案。

In [ ]:
import algviz

N = 8    # 表示棋盘的数量

viz = algviz.Visualizer(1)
t1 = viz.createTable(N, N, name="棋盘")

# row：解中每一行棋子的放置位置
# col：该列是否有棋子
# r：当前正在放置第几列
def NQueen(row, col, r):
    global sol_count
    if r == N:    # 判断基本条件
        for i in range(N):
            for j in range(N):
                t1[i][j] = ''
        for i in range(N):
            t1[i][row[i]] = '👑'
        viz.display()
        sol_count += 1
        return
    for i in range(0, N):
        if col[i] == False:    # 该列没有棋子
            conflict = False
            for j in range(0, r):
                if row[j] - i == j - r or i - row[j] == j - r:
                    conflict = True
                    break
            if not conflict:
                row[r] = i
                col[i] = True
                NQueen(row, col, r+1)
                col[i] = False

row = [-1 for _ in range(N)]
col = [False for _ in range(N)]
NQueen(row, col, 0)

# 参考链接

+ https://leetcode-cn.com/problems/n-queens/
+ https://baike.baidu.com/item/%E5%85%AB%E7%9A%87%E5%90%8E%E9%97%AE%E9%A2%98/11053477 这里面用8层循环的方式解8皇后问题，已经是很复杂了，这说明不是所有的递归都能用循环的方式展开。